## 1.Simulation a DGP where the outcome of interest depends on a randomly assigned treatment and some observed covariates.
$y_i = a*T_i+\beta'*x_i+e_i$

In [4]:
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from tqdm import tqdm
random.seed(10)

In [5]:
def fn_variance(data, ddof=0):
    n = len(data)
    mean = sum(data) / n
    return sum((x - mean) ** 2 for x in data) / (n - ddof)

In [6]:
def fn_generate_cov(dim):
    acc  = []
    for i in range(dim):
        row = np.ones((1,dim)) * corr
        row[0][i] = 1
        acc.append(row)
    return np.concatenate(acc,axis=0)

def fn_generate_multnorm(nobs,corr,nvar):

    mu = np.zeros(nvar)
    std = (np.abs(np.random.normal(loc = 1, scale = .5,size = (nvar,1))))**(1/2)
    acc = []
    for i in range(nvar):
        acc.append(np.reshape(np.random.normal(mu[i],std[i],nobs),(nobs,-1)))
    
    normvars = np.concatenate(acc,axis=1)

    cov = fn_generate_cov(nvar)
    C = np.linalg.cholesky(cov)

    Y = np.transpose(np.dot(C,np.transpose(normvars)))

    return Y

def fn_randomize_treatment(N,p=0.5):
    treated = random.sample(range(N), round(N*p))
    return np.array([(1 if i in treated else 0) for i in range(N)]).reshape([N,1])

In [58]:
def fn_generate_data(a,N,p,p0,corr,conf = True,flagX = False):
    """
    p0(int): number of covariates with nonzero coefficients
    """
    nvar = p+2 
    corr = 0.5 

    if conf==False:
        conf_mult = 0 
        
    allX = fn_generate_multnorm(N,corr,nvar)
    W0 = allX[:,0].reshape([N,1]) 
    C = allX[:,1].reshape([N,1]) 
    X = allX[:,2:] 
    
    T = fn_randomize_treatment(N) 
    err = np.random.normal(0,1,[N,1])
    beta0 = np.random.normal(5,5,[p,1])
    
    beta0[p0:p] = 0 
    Yab = a*T+X@beta0+conf_mult*0.6*C+err
    if flagX==False:
        return (Yab,T)
    else:
        return (Yab,T,X)
    
def fn_ahat_means(Yt,Yc):
    nt = len(Yt)
    nc = len(Yc)
    ahat = np.mean(Yt)-np.mean(Yc)
    se_ahat = (np.var(Yt,ddof=1)/nt+np.var(Yc,ddof=1)/nc)**(1/2)
    return (ahat,se_ahat)

def fn_bias_rmse_size(theta0,thetahat,se_thetahat,cval = 1.96):
    """
    theta0 - true parameter value
    thetatahat - estimated parameter value
    se_thetahat - estiamted se of thetahat
    """
    b = thetahat - theta0
    bias = np.mean(b)
    rmse = np.sqrt(np.mean(b**2))
    tval = b/se_thetahat 
    size = np.mean(1*(np.abs(tval)>cval))
    return (bias,rmse,size)

def fn_run_experiments(a,Nrange,p,p0,corr,conf,flagX=False):
    n_values = []
    ahats = []
    sehats = []
    lb = []
    ub = []
    for N in tqdm(Nrange):
        n_values = n_values + [N]
        if flagX==False:
            Yexp,T = fn_generate_data(a,N,p,p0,corr,conf,flagX)
            Yt = Yexp[np.where(T==1)[0],:]
            Yc = Yexp[np.where(T==0)[0],:]
            ahat,se_tauhat = fn_ahat_means(Yt,Yc)            
        elif flagX==1:
            Yexp,T,X = fn_generate_data(a,N,p,p0,corr,conf,flagX)
            Xobs = X[:,:p0]
            covars = np.concatenate([T,Xobs],axis = 1)
            mod = sm.OLS(Yexp,covars)
            res = mod.fit()
            ahat = res.params[0]
            se_ahat = res.HC1_se[0]
            
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]    
        lb = lb + [ahat-1.96*se_ahat]
        ub = ub + [ahat+1.96*se_ahat]
        
    return (n_values,ahats,sehats,lb,ub)


In [59]:
a = 2
corr = .5
conf=False
p = 5
p0 = 4 
flagX = 1
N = 1000
Y,T,X = fn_generate_data(a,N,p,p0,corr,conf,flagX)

In [60]:
data = np.concatenate([Y,T,X],axis = 1)
data = pd.DataFrame(data)
data.columns = ['self','Y', 'T', 'X1', 'X2','X3','X4']
data.to_csv('Data_1.csv')
data

,self,Y,T,X1,X2,X3,X4
0,1.152111,0.0,-0.366421,0.229596,0.597511,0.073841,-0.531055
1,-2.795042,0.0,-0.754837,-0.803859,-0.363306,0.617823,-0.638605
2,7.068557,1.0,1.798261,-0.308889,0.139340,-0.594667,0.591150
3,-21.920090,1.0,-0.571864,-1.643280,-1.696095,-1.475073,-1.123730
4,-0.226345,1.0,-0.022996,0.601482,-0.525129,0.133324,-0.938598
...,...,...,...,...,...,...,...
995,34.482473,1.0,2.451091,0.947899,1.441273,1.529525,1.118742
996,-34.933583,1.0,-1.413369,-0.555605,-2.524938,-2.383613,-1.807064
997,16.922554,1.0,1.388632,1.012010,1.602400,-0.313116,1.085435
998,-1.439401,0.0,-1.530381,0.328302,0.878239,0.232086,-0.903160


In [ ]:
import graphviz as gr

In [ ]:
g = gr.Digraph()
g.edge("T", "Y")
g.edge("X", "Y")
g

a. Do not control for any covariates

In [61]:
estDict = {}
R = 1000
for N in [100,1000]:
    ahats = []
    sehats = []
    for r in tqdm(range(R)):
        Yexp,T = fn_generate_data(a,N,5,0,corr,conf)
        Yt = Yexp[np.where(T==1)[0],:]
        Yc = Yexp[np.where(T==0)[0],:]
        ahat,se_ahat = fn_ahat_means(Yt,Yc)
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]
    estDict[N] = {
        'ahat':np.array(ahats).reshape([len(ahats),1]),
        'sehat':np.array(sehats).reshape([len(sehats),1])
    }

100%|██████████████████████████████████████| 1000/1000 [00:04<00:00, 232.63it/s]


In [62]:
a0 = a*np.ones([R,1])
for N, results in estDict.items():
    (bias,rmse,size) = fn_bias_rmse_size(a0,results['ahat'],
                                         results['sehat'])
    print(f'N={N}: bias={bias}, RMSE={rmse}, size={size}')

N=100: bias=0.001458850695198155, RMSE=0.20282026657485275, size=0.056
N=1000: bias=0.0021842589844469585, RMSE=0.06399720353965423, size=0.058


b. control for all the covariates that affect the outcome

In [63]:
estDict = {}
R = 1000
for N in [100,1000]:
    ahats = []
    sehats = []
    for r in tqdm(range(R)):
        Y,T,X = fn_generate_data(a,N,5,4,corr,conf,flagX)
        Xobs = X[:,:p0]
        covars = np.concatenate([T,Xobs],axis = 1)
        mod = sm.OLS(Y,covars)
        res = mod.fit()
        ahat = res.params[0]
        se_ahat = res.HC1_se[0]
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]
        
    estDict[N] = {
        'ahat':np.array(ahats).reshape([len(ahats),1]),
        'sehat':np.array(sehats).reshape([len(sehats),1])
    }

100%|██████████████████████████████████████| 1000/1000 [00:05<00:00, 194.11it/s]


In [64]:
a0 = a*np.ones([R,1])
for N, results in estDict.items():
    (bias,rmse,size) = fn_bias_rmse_size(a0,results['ahat'],
                                         results['sehat'])
    print(f'N={N}: bias={bias}, RMSE={rmse}, size={size}')

N=100: bias=-0.003856751018786136, RMSE=0.1504381496857265, size=0.069
N=1000: bias=-0.001248718577984226, RMSE=0.044119057711366635, size=0.051


Example in real-life situation:

For example, we want to study the relationship between years of education and hourly wage. People's hourly wage will be influenced by years of education, but also by family background and other aspects.

## 2. Simulate a DGP with a confounder
$y_i = a*T_i+0.6*Confounder+e_i$
$T = 0.6*Confounder+u$

In [44]:
def fn_generate_data_con(a,N,p,p0,corr,conf):
    """
    p0(int): number of covariates with nonzero coefficients
    """
    nvar = p+2 
    corr = 0.5 

    if conf==False:
        conf_mult = 0 # remove confounder from outcome
        
    allX = fn_generate_multnorm(N,corr,nvar)
    W0 = allX[:,0].reshape([N,1]) 
    C = allX[:,1].reshape([N,1]) 
    X = allX[:,2:] 
    T = fn_randomize_treatment(N) # choose treated units
    err = np.random.normal(0,1,[N,1])
   
    Yab = a*T+0.6*C+err
    Tab = T+0.6*C
    
    return (Yab,Tab,C)

In [65]:
a = 2
corr = .5
conf=True
p = 5
p0 = 4 
N = 1000
Y,T,C =fn_generate_data_con(a,N,p,p0,corr,conf)

In [66]:
data = np.concatenate([Y,T,C],axis = 1)
data = pd.DataFrame(data)
data.columns = ['Y', 'T', 'C']
data.to_csv('Data2.csv')
data

,Y,T,C
0,2.912345,1.242619,0.404366
1,1.375852,0.965714,1.609524
2,1.984715,1.329635,2.216058
3,2.485213,1.188121,0.313534
4,0.462530,0.115292,-1.474513
...,...,...,...
995,-0.231681,0.030666,0.051110
996,2.527723,1.383308,0.638846
997,3.395199,1.720003,1.200005
998,2.832225,1.328302,2.213837


In [ ]:
g = gr.Digraph()
g.edge("T", "Y")
g.edge("C", "Y")
g.edge("C","T")
g

a.Fail to control for the confounder

In [73]:
estDict = {}
R = 1000
for N in [100,1000]:
    ahats = []
    sehats = []
    for r in tqdm(range(R)):
        Y,T,C = fn_generate_data_con(a,N,p,p0,corr,conf=False)
        covars = np.concatenate([T],axis = 1)
        mod = sm.OLS(Y,covars)
        res = mod.fit()
        ahat = res.params[0]
        se_ahat = res.HC1_se[0]
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]
    estDict[N] = {
        'ahat':np.array(ahats).reshape([len(ahats),1]),
        'sehat':np.array(sehats).reshape([len(sehats),1])
    }

100%|██████████████████████████████████████| 1000/1000 [00:04<00:00, 209.38it/s]


In [74]:
a0 = a*np.ones([R,1])
for N, results in estDict.items():
    (bias,rmse,size) = fn_bias_rmse_size(a0,results['ahat'],
                                         results['sehat'])
    print(f'N={N}: bias={bias}, RMSE={rmse}, size={size}')

N=100: bias=-0.41295568787704634, RMSE=0.4367706130619919, size=0.878
N=1000: bias=-0.4058435816881503, RMSE=0.4195727209885637, size=0.997


b. Do control for the confounder

In [75]:
estDict = {}
R = 1000
for N in [100,1000]:
    ahats = []
    sehats = []
    for r in tqdm(range(R)):
        Y,T,C = fn_generate_data_con(a,N,p,p0,corr,conf=True)
        covars = np.concatenate([T],axis = 1)
        mod = sm.OLS(Y,covars)
        res = mod.fit()
        ahat = res.params[0]
        se_ahat = res.HC1_se[0]
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]
    estDict[N] = {
        'ahat':np.array(ahats).reshape([len(ahats),1]),
        'sehat':np.array(sehats).reshape([len(sehats),1])
    }

100%|██████████████████████████████████████| 1000/1000 [00:05<00:00, 195.27it/s]


In [76]:
a0 = a*np.ones([R,1])
for N, results in estDict.items():
    (bias,rmse,size) = fn_bias_rmse_size(a0,results['ahat'],
                                         results['sehat'])
    print(f'N={N}: bias={bias}, RMSE={rmse}, size={size}')

N=100: bias=-0.40174098282808757, RMSE=0.4304916869506998, size=0.856
N=1000: bias=-0.409915228337114, RMSE=0.4225263593164099, size=0.998


Example in real-life situation:

For example, states with higher minimum wages hire more workers and have higher employment rates. But the state of the job market, which can lead to higher minimum wages and higher employment rates, is a confounding variable.

## 3. Simulate a DGP with selection bias into the treatment
$y_i = a*T_i+e_i$
$X = 0.6*Y+0.6*T+u$

In [77]:
def fn_generate_data_bias(a,N,p,p0,corr,conf):
    """
    p0(int): number of covariates with nonzero coefficients
    """
    nvar = p+1 #1 bias
    corr = 0.5 

    allX = fn_generate_multnorm(N,corr,nvar)
    W0 = allX[:,0].reshape([N,1]) 
    C = allX[:,1].reshape([N,1]) 
    X = allX[:,2:] 
    T = fn_randomize_treatment(N) # choose treated units
    err = np.random.normal(0,1,[N,1])
    Y = a*T+err
    X = 0.6*T+0.6*Y

    
    return (Y,T,X)

In [78]:
a = 2
corr = .5
conf=True
p = 5
p0 = 4 
N = 1000
Y,T,X = fn_generate_data_bias(a,N,p,p0,corr,conf)

In [79]:
data = np.concatenate([Y,T,X],axis = 1)
data = pd.DataFrame(data)
data.columns = ['Y', 'T', 'X']
data.to_csv('Data3.csv')
data

,Y,T,X
0,3.886010,1.0,2.931606
1,-0.406508,0.0,-0.243905
2,1.279467,1.0,1.367680
3,-1.446513,0.0,-0.867908
4,0.071003,0.0,0.042602
...,...,...,...
995,2.336569,0.0,1.401942
996,2.720041,1.0,2.232025
997,0.891262,1.0,1.134757
998,0.201895,0.0,0.121137


In [ ]:
g = gr.Digraph()
g.edge("T", "X")
g.edge("T", "Y")
g.edge("X", "Y")
g.node("X", "X")
g

a. Control for the variable in between the path from cause to effect

In [85]:
estDict = {}
R = 1000
for N in [100,1000]:
    ahats = []
    sehats = []
    for r in tqdm(range(R)):
        Yexp,T,X = fn_generate_data_bias(a,N,p,p0,corr,conf)
        Yt = Yexp[np.where(T==1)[0],:]
        Yc = Yexp[np.where(T==0)[0],:]
        ahat,se_ahat = fn_ahat_means(Yt,Yc)
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]
    estDict[N] = {
        'ahat':np.array(ahats).reshape([len(ahats),1]),
        'sehat':np.array(sehats).reshape([len(sehats),1])
    }

100%|██████████████████████████████████████| 1000/1000 [00:04<00:00, 232.34it/s]


In [86]:
a0 = a*np.ones([R,1])
for N, results in estDict.items():
    (bias,rmse,size) = fn_bias_rmse_size(a0,results['ahat'],
                                         results['sehat'])
    print(f'N={N}: bias={bias}, RMSE={rmse}, size={size}')

N=100: bias=0.004628638075049398, RMSE=0.2056716688581379, size=0.056
N=1000: bias=-0.004400156693251745, RMSE=0.06353894142357944, size=0.049


In [87]:
estDict = {}
R = 1000
for N in [100,1000]:
    ahats = []
    sehats = []
    for r in tqdm(range(R)):
        Y,T,X = fn_generate_data_bias(a,N,p,p0,corr,conf)
        Xobs = X[:,:p0]
        covars = np.concatenate([T,Xobs],axis = 1)
        mod = sm.OLS(Y,covars)
        res = mod.fit()
        ahat = res.params[0]
        se_ahat = res.HC1_se[0]
        ahats = ahats + [ahat]
        sehats = sehats + [se_ahat]
        
    estDict[N] = {
        'ahat':np.array(ahats).reshape([len(ahats),1]),
        'sehat':np.array(sehats).reshape([len(sehats),1])
    }

100%|██████████████████████████████████████| 1000/1000 [00:05<00:00, 195.80it/s]


In [88]:
a0 = a*np.ones([R,1])
for N, results in estDict.items():
    (bias,rmse,size) = fn_bias_rmse_size(a0,results['ahat'],
                                         results['sehat'])
    print(f'N={N}: bias={bias}, RMSE={rmse}, size={size}')

N=100: bias=-3.0, RMSE=3.0, size=1.0
N=1000: bias=-3.0, RMSE=3.0, size=1.0


Example in real-life situation:

For example, age has a direct impact on job satisfaction, but at the same time age affects income, and income affects job satisfaction. Here, X is income, T is age, and Y is job satisfaction.